In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import lsq_linear
import sympy as sym

In [2]:
def clean_data_long(filename: str, spaces: bool, messy: bool):
    df = pd.read_csv(filename, skipinitialspace=spaces)
    females = "F"
    if spaces:
        females += " "
    df = df[df["Sex"] == females] # remove male cattles
    
    df["Number of Cattle"] = df["Number of Cattle"].str.replace(',', '').astype(int) # convert strings to ints
    old_group_num = df.groupby("Age Groups").sum()["Number of Cattle"]
    
    if messy:
        old_group_num.drop(old_group_num.index[0], inplace=True)
        old_group_num.loc['36 to 42 months '] += old_group_num.loc['366 to 42 months ']
        old_group_num.drop('366 to 42 months ', inplace=True)
    age_groups = old_group_num.index.tolist()
    
    # change order of months names
    to_remove = "6 to 12 months"
    if spaces:
        to_remove += " "
    age_groups.remove(to_remove)
    age_groups[1] = to_remove
    last_group = "102+months"
    if spaces:
        last_group += " "
    age_groups.append(last_group)
    
    # unite months groups to year groups
    year_nums = []
    for i in range(0, len(age_groups), 2):
        year_nums.append(old_group_num.loc[age_groups[i]] + old_group_num.loc[age_groups[i+1]])

    return pd.Series(year_nums)

data_2015 = clean_data_long("data/great-britain-cattle-population-england-on_1-january-2015.csv", True, False)
data_2014 = clean_data_long("data/great-britain-cattle-population-on_1-january-2014.csv", True, True)
data_2013 = clean_data_long("data/gb-cattle-population-on-01-january-2013.csv", False, False)

In [3]:
def clean_data_short(filename, to_add=0):
    df = pd.read_csv(filename, skiprows=lambda x: x <= (1+to_add) or x > (21+to_add), usecols=[0,1,2])

    df["Number of Cattle"] = df["Dairy"].str.replace(',', '').astype(int) \
        + df["Non Dairy"].str.replace(',', '').astype(int)

    df = df.drop(18) # remove total for all ages

    age_groups = df["Number of Cattle"]
    year_nums = []

    for i in range(0, df.shape[0], 2):
        year_nums.append(age_groups.loc[i] + age_groups.loc[i+1])

    return pd.Series(year_nums)

data_2012 = clean_data_short("data/great-britain-cattle-population-on_1-january-2012.csv")
data_2011 = clean_data_short("data/great-britain-cattle-population-on_1-january-2010-to-1-january-2011.csv", 24)
data_2010 = clean_data_short("data/great-britain-cattle-population-on_1-january-2010-to-1-january-2011.csv")

In [4]:
data_2010_without_last = data_2010[:-1]
data_2011_without_first = data_2011[1:].reset_index(drop=True)
survival_rates = data_2011_without_first.div(data_2010_without_last)
survival_rates

0    0.887726
1    0.626557
2    0.802723
3    0.868029
4    0.846224
5    0.821675
6    0.798267
7    3.286115
dtype: float64

The last rate is greater than 0 because the Leslie model assumes that there is no chance of survival in the last age group, but our dataset unites several last years in 1 group, which makes its size much bigger compared to others.

In [5]:
n_new = np.array([data_2011[0], data_2012[0], data_2013[0], data_2014[0]])
n_new

array([1289890, 1302637, 1289721, 1258521])

In [6]:
matrix_old = []
for data in (data_2010, data_2011, data_2012, data_2013):
    matrix_old.append(data.to_numpy())
matrix_old = np.asarray(matrix_old)
matrix_old 

array([[1258186, 1168933,  711992,  561177,  481494,  425566,  363971,
         306653,  990408],
       [1289890, 1116925,  732403,  571532,  487118,  407452,  349677,
         290546, 1007697],
       [1302637, 1116825,  692592,  589895,  484179,  400816,  324702,
         268591,  938977],
       [1289721, 1147331,  713191,  568217,  496199,  393830,  314057,
         243883,  714561]])

In [7]:
fertility_rates = lsq_linear(matrix_old, n_new, bounds=(0, np.inf)).x
fertility_rates

array([0.4324738 , 0.18287974, 0.2421186 , 0.17955221, 0.15218551,
       0.05459854, 0.03664139, 0.00922788, 0.14757397])

In [13]:
def nnls(C, d):
    '''Linear least squares with nonnegativity constraints.
    returns tuple (x, resnorm, residual) the vector x that minimizes norm(C*x - d)
    subject to x >= 0
    '''

    eps = 2.22e-16
    tol = 10 * eps * np.linalg.norm(C,1) * (max(C.shape)+1)
    C = np.asarray(C)
    (m,n) = C.shape
    P = []
    R = [x for x in range(0,n)]

    x = np.zeros(n)
    resid = d - np.dot(C, x)
    w = np.dot(C.T, resid)

    count = 0

    while np.any(R) and np.max(w) > tol:

        j = np.argmax(w)
        P.append(j)
        R.remove(j)

        AP = np.zeros(C.shape)
        AP[:,P] = C[:,P]

        s=np.dot(np.linalg.pinv(AP), d)

        s[R] = 0
     
        while np.min(s) < 0:
            i = [i for i in P if s[i] <= 0]

            alpha = min(x[i]/(x[i] - s[i]))
            x = x + alpha*(s-x)

            j = [j for j in P if x[j] == 0]
            if j:
                R.append(*j)
                if j in P:
                    P.remove(j)
            
            AP = np.zeros(C.shape)
            AP[:,P] = C[:,P]
            s=np.dot(np.linalg.pinv(AP), d)
            s[R] = 0
     
        x = s
        resid = d - np.dot(C, x)

        w = np.dot(C.T, resid)

    return (x, sum(resid * resid), resid)

In [14]:
fr = nnls(matrix_old, n_new)
fr

(array([0.55412103, 0.23432335, 0.22560577, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.1596967 ]),
 3.010287331906003e-16,
 array([-8.84756446e-09, -8.38190317e-09, -8.61473382e-09, -8.84756446e-09]))

In [10]:
leslie_matrix = np.zeros((9, 9))
leslie_matrix[0] = fr[0]
for i in range(len(survival_rates)):
    leslie_matrix[1+i][i] = survival_rates[i]
leslie_matrix

array([[0.55412103, 0.23432335, 0.22560577, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.1596967 ],
       [0.88772646, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.62655687, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.8027225 , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.86802916, 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.84622446,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.82167513, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.79826689, 0.        , 0.        ],


In [11]:
found_data_2015 = leslie_matrix @ data_2014
found_data_2015

array([1242794.11512353, 1117222.38836309,  719896.2983473 ,
        597437.46060068,  510875.09008388,  410551.72019589,
        337754.49380825,  253331.59562712,  806544.04711514])

In [15]:
parameters = np.array([[45354370292635865200 / 47302965675958935147, - 105902580216774627548 / 47302965675958935147, - 168500656198419124033 / 47302965675958935147, - 229658469253746953843 / 47302965675958935147, - 1292329653957815307833 / 47302965675958935147, 206380777974865284079/47302965675958935147], \
                      [5814265095302702528 / 47302965675958935147, - 36281030357732475586 / 47302965675958935147, - 72231305199053088041 / 47302965675958935147, - 85661161750306210042 / 47302965675958935147, - 438373026017808338296 / 47302965675958935147, 80678546593087218692/47302965675958935147], \
                      [-5710787771140112144/47302965675958935147, - 12923750724465337538/47302965675958935147, - 7985451765899301886/47302965675958935147, - 5764709626874342204/47302965675958935147, 42310959838267308091/47302965675958935147, 4327268647624226713/47302965675958935147], \
                      [-36315859153530022997/47302965675958935147, 63773051392654103710/47302965675958935147, 109764656866129470599/47302965675958935147, 144733827639295973356/47302965675958935147, 747930634193780457829/47302965675958935147, - 93230484454458235811/47302965675958935147]])

In [16]:
parameters

array([[  0.95880606,  -2.23881481,  -3.56215839,  -4.85505435,
        -27.32026704,   4.36295642],
       [  0.12291545,  -0.76699272,  -1.52699316,  -1.81090468,
         -9.26734761,   1.70557058],
       [ -0.1207279 ,  -0.27321227,  -0.16881503,  -0.12186783,
          0.89446738,   0.09147986],
       [ -0.76772901,   1.34818294,   2.32046036,   3.05971995,
         15.81149561,  -1.97092261]])